In [8]:
import os
import numpy as np
from tqdm import tqdm
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, default_data_collator, get_scheduler

from utils import _preprocess_training, _preprocess_validation, train, eval


MAX_LENGTH = 384
STRIDE = 128
NUM_TRAIN_EPOCHS = 20
N_BEST = 20
MAX_ANSWER_LENGTH = 30
TRAINED_MODEL_PATH = './models'
os.makedirs(TRAINED_MODEL_PATH, exist_ok=True)

### Обучение

In [9]:
# Загрузим датасет и посмотрим на формат данных
dataset = load_dataset("sberquad")
dataset['train'][0]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

In [10]:
# загрузка токенизатора и необученной модели
model = AutoModelForQuestionAnswering.from_pretrained("cointegrated/rubert-tiny2")
model.cuda();
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [11]:
# предобработка трейн части датасета
def preprocess_training(data):
    questions = [q.strip() for q in data["question"]]
    inputs = tokenizer(
        questions,
        data["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    inputs = _preprocess_training(data, inputs)
    return inputs

train_data = dataset["train"].map(
    preprocess_training,
    batched=True,
    remove_columns=dataset["train"].column_names,
)
train_data.set_format("torch")

In [12]:
# предобработка валидационной части датасета
def preprocess_validation(data):
    questions = [q.strip() for q in data["question"]]
    inputs = tokenizer(
        questions,
        data["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    inputs = _preprocess_validation(data, inputs)
    return inputs


validation_data = dataset["validation"].map(
    preprocess_validation,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)
val_data = validation_data.remove_columns(["example_id", "offset_mapping"])
val_data.set_format("torch")

In [13]:
# загрузка в торчевские далатоадеры
train_dataloader = DataLoader(
    train_data,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=120,
)
eval_dataloader = DataLoader(
    val_data,
    shuffle=False,
    collate_fn=default_data_collator,
    batch_size=120,
)

optimizer = AdamW(model.parameters(), lr=2e-5)
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=1,
    num_training_steps=NUM_TRAIN_EPOCHS * len(train_dataloader),
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# и обучим на sbersquad
train(model, train_dataloader,
      eval_dataloader, validation_data, dataset["validation"],
      optimizer, lr_scheduler, N_BEST, MAX_ANSWER_LENGTH,
      NUM_TRAIN_EPOCHS, TRAINED_MODEL_PATH)

### Тест

In [14]:
# Протестируем свежеобученную модель
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModelForQuestionAnswering.from_pretrained(TRAINED_MODEL_PATH)
model.eval()
model.cuda();

In [17]:
# посмотрим метрики обученной модели на валидации
eval(model, eval_dataloader, validation_data, dataset["validation"], N_BEST, MAX_ANSWER_LENGTH, return_scores=True)

100%|██████████| 5036/5036 [00:05<00:00, 844.36it/s]


{'exact_match': 35.02779984114377, 'f1': 55.33552449242349}

In [7]:
# загрузим тестовые данные и посмотрим результат сами
test_dataset = dataset["test"].map(
    preprocess_validation,
    batched=True,
    remove_columns=dataset["test"].column_names,
)
test_data = test_dataset.remove_columns(["example_id", "offset_mapping"])
test_data.set_format("torch")

test_loader = DataLoader(
    test_data,
    shuffle=False,
    collate_fn=default_data_collator,
    batch_size=10,
)

results = eval(model, test_loader, test_dataset, dataset["test"], N_BEST, MAX_ANSWER_LENGTH, return_scores=False)
results = iter(results)

100%|██████████| 23936/23936 [00:27<00:00, 880.67it/s]


In [11]:
# Можем поитерироваться по результатам и визуально оценить как оно работает
# к сожалению скор можно получить только на валидации, в тестовой выборке нет ответов
next(results)

('Многоклеточный организм — внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых (кроме стволовых, например, клеток камбия у растений) дифференцированы, то есть различаются по строению и выполняемым функциям. Следует отличать многоклеточность и колониальность. У колониальных организмов отсутствуют настоящие дифференцированные клетки, а следовательно, и разделение тела на ткани. Граница между многоклеточностью и колониальностью нечёткая. Например, вольвокс часто относят к колониальным организмам, хотя в его колониях есть чёткое деление клеток на генеративные и соматические. Кроме дифференциации клеток, для многоклеточных характерен и более высокий уровень интеграции, чем для колониальных форм. Многоклеточные животные, возможно, появились на Земле 2,1 миллиарда лет назад, вскоре после кислородной революции .',
 'Как называется внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых 